## Data ETL 

### Initial data - dictionaries

I asked my friends to provide data in form of a dictionary to ensure correct data structure. Not all of them are coders, but all of them encountered data in life before. I explained that the dictionary is a data structure that for every category (key) has a corresponding value. I asked for a dictionary to look like this:

In [ ]:
my_name = {
    'nickname': '', # Your in-game summoner name
    'tagline':'', # Riot_id tagline
    'role': '', # The role you are gonna play
    'champ_pool': ['champ1', 'champ2', 'champ3', ...] # List you favorite champs but please no more than 6
}

They were not so sure about it and some asked if they can just send a table in some popular program (that should remain nameless). But with my mission of disenchanting coding for people I proposed that I will fill the dictionary based on my data and that they could simply repeform and then they agreed. I knew they could do it!

#### My example

In [ ]:
julian_dict = {
    'nickname': 'Vile Vilainess',
    'tagline':'JNX',
    'role': 'jungler',
    'champ_pool': ['Lilia', 'Volibear', 'Sejuani', 'Poppy', 'Ekko', 'Hecarim'] # I know I am already frustrated with myself that i set limit to 6
}

### My friends Data



As a result of the process I have been provided with the `npb_data.json`file that contains all required data of the Nexus Park Boys team. 

In [ ]:
import json 
import pandas as pd

# Path to a file
file_path = 'npb_data.json'

with open(file_path, 'r') as file:
    json_data = json.load(file)


npb_df = pd.DataFrame.from_dict(json_data, orient='index')

## Creating and population an sqlite database

### Creation of a DB and establishing a connection

In [ ]:
import sqlite3 as sql

# Create SQLite database
main_db = sql.connect(r'project_database.db')

# Create an explicit cursor object since SQLite embeeded in Python does not support implicit cursor object
cursor = main_db.cursor()

### Inserting prepared data into a database
We create new table for each prepared dataframe 

In [ ]:
npb_df = npb_df.iloc[:, 0:2]

# Insert data from DataFrame into new table in DB
npb_df.to_sql('players', main_db, if_exists='replace', index = True)
champion_pool.to_sql('champion_pool', main_db, if_exists='replace', index = True)

npb_df

## Getting champions ID

Our team obviously operates with champion names but what I know is that Riot API operates with champion IDs. HAving that in mind i got [json file](champions_data.json) from Riot's site for developers that will allow me to connect (relate) champion names to theirs ID. 

#

In [ ]:
file_path = 'champions_data.json'

with open (file_path, 'r') as file:
    data = json.load(file)


# Extract relevant data for DataFrame
champion_list = []
for champion, details in data['data'].items():
    champ_dict = {
        'id': details['id'],
        'name': details['name'],
        'title': details['title'],
        'attack': details['info']['attack'],
        'defense': details['info']['defense'],
        'magic': details['info']['magic'],
        'difficulty': details['info']['difficulty'],
        'tags': ", ".join(details['tags']),
        'partype': details['partype'],
        'hp': details['stats']['hp'],
        'hpperlevel': details['stats']['hpperlevel'],
        'mp': details['stats']['mp'],
        'mpperlevel': details['stats']['mpperlevel'],
        'movespeed': details['stats']['movespeed'],
        'armor': details['stats']['armor'],
        'armorperlevel': details['stats']['armorperlevel'],
        'spellblock': details['stats']['spellblock'],
        'spellblockperlevel': details['stats']['spellblockperlevel'],
        'attackrange': details['stats']['attackrange'],
        'hpregen': details['stats']['hpregen'],
        'hpregenperlevel': details['stats']['hpregenperlevel'],
        'mpregen': details['stats']['mpregen'],
        'mpregenperlevel': details['stats']['mpregenperlevel'],
        'crit': details['stats']['crit'],
        'critperlevel': details['stats']['critperlevel'],
        'attackdamage': details['stats']['attackdamage'],
        'attackdamageperlevel': details['stats']['attackdamageperlevel'],
        'attackspeedperlevel': details['stats']['attackspeedperlevel'],
        'attackspeed': details['stats']['attackspeed']
    }
    champion_list.append(champ_dict)

# Create DataFrame
champions_df = pd.DataFrame(champion_list)

Now simply add another table into our db

In [ ]:
champions_df.to_sql('champions', main_db, if_exists='replace', index=True)

### Issue with creating sql tables using Pandas

There is no way to create tables with primary key directly from Python. Furthermore SQLite does not support altering a table by adding a primary key. 

An ideal approach would be to first create a tables in SQL and this is an approach that will be generally used in this repository, however just to simulate a real life problem and that any problem can be solved we will execute SQL query using temporary tables to get a tables with primary key. 